# Week 02 Assignment glucose level data


Missing data occurs commonly in many data applications. Especially wearable sensor devices encounter measurement errors resulting in missing data. Although pandas can handle missing data, for instance, to exclude missing data in the descriptive analysis, we might decide to reshape our data to improve quality. We might consider excluding certain rows, columns, or decide to impute our data if we take an argumentative approach. This week we will work with sensor data of a semester 01 project. The data is timeseries related but a lot of sensor errors occurred. First, we inspect the quality data. Then we will work with imputation, interpolation, and smoothing techniques to overcome this issue of the error readings. Finaly we plot the data. Visualization is a method to translate data into information. As a data scientist you should be able to choose an appropiate visualizations. 


Keywords: data loading, data inspection, data exploration, data cleaning, impute data, missing data, timeseries, pandas, visualization, interactive plots, overflow/underflow error

More to read
- https://fennaf.gitbook.io/bfvm22prog1/
- https://towardsdatascience.com/how-to-analyze-blood-glucose-data-with-python-data-science-packages-4f160f9564be


More about pandas and bokeh programming
- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.interpolate.html
- https://rubberduckdebugging.com/
- https://docs.bokeh.org/en/latest/docs/gallery.html


Learning objectives

- Further practice numpy / pandas
- Read, inspect, clean, and impute JSON data. Handle missing data
- Apply different interpolation/smoothing techniques
- Recognize overflow/underflow error
- Perform visual and statistical analysis for time series data
- Maintain development environment 
- Apply coding standards and FAIR principles


Note: you can bring your own sensor data, but for the practical you need some missing data points. 

By the end of this week the student can:

- load a json dataset 
- typecast the Pandas DataFrame to appropiate data types
- inspect the dataset for quality and metadata information
- add a column with interpolated data in Pandas DataFrame
- perform visual analysis

please add your own topics you want to learn here: https://padlet.com/ffeenstra1/69vbvy0l8t32rq55


The assignment consists of 6 parts:

- [part 1: load the data](#0)
     - [Exercise 1.1](#ex-11)
- [part 2: prepare for inspection](#1)
     - [Exercise 2.1](#ex-21)
- [part 3: inspect the data](#2)
     - [Exercise 3.1](#ex-31)
- [part 4: interpolate the data](#3)
     - [Exercise 4.1](#ex-41)
- [part 5: visualize the data](#4)
     - [Exercise 5.1](#ex-51)
- [part 6: Challenge](#5)
     - [Exercise 6.1](#ex-61)

Part 1 and 5 are mandatory, part 6 is optional (bonus)
Mind you that you cannot copy code without referencing the code. If you copy code you need to be able to explain your code verbally and you will not get the full score. 


<a name='0'></a>
## Part 1: Load the data

Instructions: Load the json datafile `glucose.json` into a pandas dataframe. Check your dataframe with a `.head()` to compare with the expected outcome. 

Preferably we read the data not with a hard coded data path but using a config file. See https://fennaf.gitbook.io/bfvm22prog1/data-processing/configuration-files/yaml

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>json.load() method reads a file, pd.read_json converts it to a Pandas DataFrame</li>
    <li>when loading into a Pandas DataFrame use records orientation </li>
</ul>
</details>

<a name='ex-11'></a>
### Code your solution

In [34]:
# CODE YOUR SOLUTION HERE
import yaml
import json
import pandas as pd


#parse a config yaml file to a config json file 
with open('config.yaml', 'r') as yaml_config_file:
    configuration = yaml.safe_load(yaml_config_file)

with open('config.json', 'w') as json_config_file:
    json.dump(configuration, json_config_file)

#read the glucosse file using config.json 
with open(str(json.load(open('config.json'))['glucose_path'])) as glucose_reader:
    glucose_data =json.load(glucose_reader)

df = pd.read_json(glucose_data)
df_copy=df.copy()
df.head()





,ID,time,recordtype,glucose
0,2845.0,2019-04-25 00:08,1,109
1,2850.0,2019-04-25 00:50,1,
2,2877.0,2019-04-25 07:02,1,123
3,2881.0,2019-04-25 07:34,1,158
4,2886.0,2019-04-25 08:19,1,


#### Expected outcome: 

<a name='1'></a>
## Part 2: Prepare the data

Check the datatypes of your dataframe. The `glucose` field should be an integer, the `time` field should have a datetime format. If the datatypes are different you should typecast them to the right format.
Make sure that your dataset is sorted by the time column


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use astype() method or pandas.DataFrame.to_datetime() for instance</li>
    <li>make sure that the empty spaces are filled with NaN. Use errors='coerce'</li>
    <li>set_index(), sort_index() and reset_index() are helpful to sort on index</li>
</ul>
</details>

<a name='ex-21'></a>
### Code your solution

In [35]:

#typecasting ID, glucose and time columns 
df['ID'] = df['ID'].astype(int)
df['glucose'] = df['glucose'].apply(pd.to_numeric, errors='coerce')
df['time'] = df['time'].apply(pd.to_datetime, errors='coerce')
print(df.dtypes) 

#sort the dataset by the time column
df = df.set_index(['time'])
df=df.sort_index(ascending=True)
print('\nDataFrame after setting index on time column and sorting by index\n')
print(df)

#to keep a copy of last modified dataframe 
df_copy =df.copy()

ID                     int32
time          datetime64[ns]
recordtype             int64
glucose              float64
dtype: object

DataFrame after setting index on time column and sorting by index

                             ID  recordtype  glucose
time                                                
2019-04-25 00:08:00        2845           1    109.0
2019-04-25 00:14:00 -2147483648           0      NaN
2019-04-25 00:29:00 -2147483648           0      NaN
2019-04-25 00:44:00 -2147483648           0      NaN
2019-04-25 00:50:00        2850           1      NaN
...                         ...         ...      ...
2019-04-25 23:02:00 -2147483648           0      NaN
2019-04-25 23:18:00 -2147483648           0      NaN
2019-04-25 23:31:00        3062           1    111.0
2019-04-25 23:33:00 -2147483648           0      NaN
2019-04-25 23:48:00 -2147483648           0      NaN

[136 rows x 3 columns]


#### Expected outcome: 

<a name='2'></a>
## Part 3: Inspect the data

Now that we prepared the data we are going to inspect the data to get more familiar with the data. You can answer the questions below but feel free to explore the data further by plotting and or statistics, group by overviews and so on. 

To be answered at least:
- what is the quantity of the data (observations and features)
- what is the percentage missing data for glucose?
- is there a specific pattern for glucose value and recordtype?
- what is the time range?

Code the solutions to your answers. Create meaningful overviews or statistics

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>In the week 01 assignment some functions were explained to inspect missing values</li>
    <li>In the week 01 assignment some functions were explained to groupby value</li>
</ul>
</details>

<a name='ex-31'></a>
### Code your solution

In [36]:
#CODE YOUR SOLUTION HERE
df = df.reset_index()

#quantity of data (observation and features) 
#after run, We can see that ID, time, and recordtype hav no zero values,
#whereas glucozse show a plenty, nearly half of the rows.
print(f'\n {df.info( )}') 
print(f'\n {df.describe( )}')


#the percentage missing data for the glucose
glucose_missData_percent=df['glucose'].isna().sum()/(df['glucose'].count()+df['glucose'].isna().sum())*100
print (f"\nthe pecentage missing data for glucose is {glucose_missData_percent}%" ) 

#time range
print (f"\nthe time range is from {df['time'].min()} to {df['time'].max()}" ) 

#to see if there is a specific pattern for glucose and recordtype. the result are exported into 
#csv "recordtype_grouped.csv'file
indexed_sorted_df=pd.DataFrame(df.set_index(['recordtype', 'glucose']), ).sort_index()
pd.DataFrame(indexed_sorted_df.groupby(['recordtype'])).to_csv('recordtype_grouped.csv')

#After inspecting 'recordtype_grouped.csv' file, containing dataframe grouped by recordtype column,
#it can bee seen that in  all records with datatype zero, glucose value has been missed.
#Furthermore, for almost all records with recordtype=1,  except two, glucose values have been reported.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   time        136 non-null    datetime64[ns]
 1   ID          136 non-null    int32         
 2   recordtype  136 non-null    int64         
 3   glucose     52 non-null     float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(1)
memory usage: 3.8 KB

 None

                  ID  recordtype     glucose
count  1.360000e+02  136.000000   52.000000
mean  -1.294805e+09    0.397059  123.403846
std    1.054625e+09    0.491097   20.734796
min   -2.147484e+09    0.000000   86.000000
25%   -2.147484e+09    0.000000  109.750000
50%   -2.147484e+09    0.000000  121.500000
75%    2.949500e+03    1.000000  129.000000
max    3.062000e+03    1.000000  184.000000

the pecentage missing data for glucose is 61.76470588235294%

the time range is from 2019-04-25 00:08:00 to 2019-04-25 23:4

C:\Users\Hedieh\AppData\Local\Temp\ipykernel_10704\3107539342.py:21: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  pd.DataFrame(indexed_sorted_df.groupby(['recordtype'])).to_csv('recordtype_grouped.csv')


After inspecting 'recordtype_grouped.csv' file, containing dataframe grouped by recordtype column it can bee seen that in  all records with datatype zero, glucose value has been missed.Furthermore, for almost all records with recordtype=1,  except two, glucose values have been reported.

<a name='3'></a>
## Part 4: Interpolate the data

A lot of data is missing. Use interpolation to fill the missing values. Create a new column with the interpolated data. Take an argumentative approach. Select an interpolation method that suits the nature of the data and explain your choice. Mind you that the expected outcome of the interpolation values can differ from the example below

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use Pandas.DataFrame.interpolate() method</li>
</ul>
</details>

<a name='ex-41'></a>
### Code your solution

In [37]:
#CODE YOUR SOLUTION HERE

#this copy containes time  converted   to datetime format and  has been set  as the index of the dataframe
df=df_copy

#fill glucose missing values by linera interpolate method
df['interpolated'] = df['glucose'].interpolate(method='linear')
print("\n glucose missing value filled by linear interpolate method before emoving related rows to the negative ID column\n")
print(df.head())

#after ispecting the data, We can see that there is a negative ID value,
#with too many NaN values, which is removed along with its related rows,
#to yield the better result for interpolation 

#to undo the last changes of dataframe
df=df.drop(['interpolated'], axis=1)

#to remove negative ID value as well as its related rows. 
df=df.drop(df[df['ID'] < 0].index)

######################################################################
#to examine different methods of interpolation and compare the tesults
######################################################################
#fill glucose missing value by linera interpolate method
df['linear_interpolated'] = df['glucose'].interpolate(method='linear')
print("\n glucose missing value filled by linear interpolate method after removing related rows to the negative ID column \n")
print(df.head())



#fill glucose missing value by pad interpolate method
df['pad_interpolated'] = df['glucose'].interpolate(method='pad', limit=2)
print("\n glucose missing value filled by pad interpolate method \n")
print(df.head())

#fill glucose missing value by polynomial interpolate method
df['polynomial_interpolated'] = df['glucose'].interpolate(method='polynomial',order=2)
print("\n glucose missing value filled by polynomial interpolate method \n")
print(df.head())

#fill glucose missing value by spline interpolate method
df['spline_interpolated'] = df['glucose'].interpolate(method='spline',order=2)
print("\n glucose missing value filled by spline interpolate method \n")
print(df.head())


 glucose missing value filled by linear interpolate method before emoving related rows to the negative ID column

                             ID  recordtype  glucose  interpolated
time                                                              
2019-04-25 00:08:00        2845           1    109.0    109.000000
2019-04-25 00:14:00 -2147483648           0      NaN    109.466667
2019-04-25 00:29:00 -2147483648           0      NaN    109.933333
2019-04-25 00:44:00 -2147483648           0      NaN    110.400000
2019-04-25 00:50:00        2850           1      NaN    110.866667

 glucose missing value filled by linear interpolate method after removing related rows to the negative ID column 

                       ID  recordtype  glucose  linear_interpolated
time                                                               
2019-04-25 00:08:00  2845           1    109.0                109.0
2019-04-25 00:50:00  2850           1      NaN                116.0
2019-04-25 07:02:00  2877   

#### Example outcome

<a name='4'></a>
## Part 5: Plot the data

Create a plot with the original data and the interpolated data. Consider what the best representation is for visualisation of actual values and modelled/imputed values. Provide argumentation for the plot of choice. 

Mind you, there are several types of plots. The most used are: 
- Compare values->	Bar, boxplot, violin plot, scatter, line. 
- Show composition-> Stacked bar, pie, mosaic, area, burtin.
- Show distribution-> 	Histogram, density, qqplot, bubble. 
- Analyzing trend-> Line, bar, dual axis 
- Relations-> Scatter, heatmap


### Code your solution

In [38]:
from bokeh.io import output_notebook
from bokeh.plotting import figure,output_file, show
from bokeh.layouts import gridplot
from bokeh.io import output_notebook
from bokeh.plotting import ColumnDataSource
output_notebook()

Loading BokehJS ...

In [39]:
from bokeh.plotting import figure,gridplot, show
from bokeh.models import DatetimeTickFormatter, NumeralTickFormatter
#########################################
#data preperation for plots
#########################################
df = df.reset_index()

#to get the time part of the datetime datatype
x=df['time'].dt.time

#to get the values of y axis
y=[df['linear_interpolated'].values.tolist(),df['pad_interpolated'].values.tolist(),df['polynomial_interpolated'].values.tolist(),df['spline_interpolated'].values.tolist()]

#to get the  diagrams' color
colors=["red","#9ebcda","#31a354","#ffeda0","#fc9272"]

#to get the  diagrams' label
leg_label=['linear','pad','polynomial','spline']

# create a new plot with a title and axis labels
p = figure(title="Multiple Interpolate Methodes on Glucose", x_axis_label="time",x_axis_type="datetime", y_axis_label="glucose")

# add multiple renderers
p.line(x, df['glucose'].values.tolist(), legend_label="non-interpolated_glucose.", color="red", line_width=2)

axis_indx=0
for axis_indx in range(len(y)):
       p.scatter(x, y[axis_indx], legend_label=leg_label[axis_indx], color=colors[axis_indx], line_width=2)
      


#to display axis in a appropriate format
p.yaxis.formatter = NumeralTickFormatter(format="0.00")

# show the plots
show(p)

In [40]:
#According the plots it seems that pad and linear interpolating methods, both, suits the data beter.

<a name='6'></a>
## Part 6: Challenge

It might even be interesting to introduce a widget in which you can select different methods to interpolate.
1. Can you improve the interpolation by choosing an other method?
2. Can you add an rolling mean line? 
2. Can you improve the plot by making it interactive?

<a name='ex-61'></a>
### Code your solution

In [41]:

from bokeh.io import show

#calculate rolling mean of  3 former values  for glocuse
df['rolling_glucose_3'] = df['linear_interpolated'].rolling(3).mean() 

#filter to view a subset of columns 
print(df.loc[:,['time', 'ID', 'linear_interpolated','rolling_glucose_3' ]].head(10))

#visualize the glucose compared to the rolling mean 
#define the  plot
x=df['time'].dt.time
y =[df['linear_interpolated'].values.tolist(),df['rolling_glucose_3'].values.tolist()] 

leg_lable=['glucose','rolling mean']
colors=['red','green'] 
# create a new plot with a title and axis labels
p = figure(title="The glucose interpolated by linear regresion compared to the rolling mean", x_axis_label="time",x_axis_type="datetime", y_axis_label="glucose")
axis_indx=0
for axis_indx in range(len(y)):
       p.line(x, y[axis_indx], legend_label=leg_lable[axis_indx], color=colors[axis_indx], line_width=2)
      
#show the plots
show(p)     

                 time    ID  linear_interpolated  rolling_glucose_3
0 2019-04-25 00:08:00  2845                109.0                NaN
1 2019-04-25 00:50:00  2850                116.0                NaN
2 2019-04-25 07:02:00  2877                123.0         116.000000
3 2019-04-25 07:34:00  2881                158.0         132.333333
4 2019-04-25 08:19:00  2886                154.5         145.166667
5 2019-04-25 13:14:00  2909                151.0         154.500000
6 2019-04-25 14:17:00  2916                129.0         144.833333
7 2019-04-25 14:45:00  2922                161.0         147.000000
8 2019-04-25 14:55:00  2925                184.0         158.000000
9 2019-04-25 14:59:00  2927                178.0         174.333333


In [73]:
from bokeh.models import  ColumnDataSource,CustomJS, RadioGroup
from bokeh.layouts import column,row
from bokeh.plotting import figure, show

#######################################################################################
#visualizing the glucose interpolate using an interactive plot:
#######################################################################################

# Create plots as p
x=df['time'].dt.time
y=[df['glucose'].values.tolist(),df['linear_interpolated'].values.tolist(),df['pad_interpolated'].values.tolist(),df['polynomial_interpolated'].values.tolist(),df['spline_interpolated'].values.tolist()]
source = ColumnDataSource(data=dict(x=x, y=y[0])) 
 

#plot the static diagram
p = figure(title="the glucose interpolated by linear regresion compared to the rolling mean", x_axis_label="time",x_axis_type="datetime", y_axis_label="glucose")
p.line('x', 'y', source=source,  line_width=2, legend_label="glucose", color="red")

#create a radio group object
LABELS = ['linear','pad','polynomial','spline']

radio_group = RadioGroup(labels=LABELS, active=0)

# plot the static diagram according the value of radio-button
def radiogroup_click(attr,old,new):
        #Getting radio button value
        active_radio=radio_group.active
        if active_radio==0:
            source = ColumnDataSource(data=dict(x=x, y=y[1]))
            leg_label='linear'  
        elif active_radio==1:
            source = ColumnDataSource(data=dict(x=x, y=y[2]))
            leg_label='pad'
        elif active_radio==2:
            source = ColumnDataSource(data=dict(x=x, y=y[3])) 
            leg_label='polynomial'      
        else:
            source = ColumnDataSource(data=dict(x=x, y=y[4]))
            leg_label='spline'
        p.line('x', 'y', source=source,  line_width=2, legend_label=leg_label, color="green")     
radio_group.on_change("active",radiogroup_click)

# Arrange plots and radiogroup in layouts
layout = row(radio_group, column(p))
show(layout)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



In [71]:
from bokeh.models import ColumnDataSource, Slider, CustomJS
from bokeh.layouts import column,row
from bokeh.plotting import figure, show
#######################################################################################
# visualize the glucose compared to the rolling mean of it using an interactive plot:
#######################################################################################

# Create plots as p
x=df['time'].dt.time
y= df['linear_interpolated'].rolling(3).mean() 
source = ColumnDataSource(data=dict(x=x, y=y)) 

#plot the static diagram
p = figure(title="the glucose interpolated by linear regresion compared to the rolling mean", x_axis_label="time",x_axis_type="datetime", y_axis_label="glucose")
p.line(x=df['time'].dt.time, y=df['linear_interpolated'].values.tolist(), line_width=2, legend_label="glucose", color="red")
# Create Slider object
sld = Slider(start=2, end=10, value=2, step=1, title="rolling mean number")

# Create a new dynamic plot 



sld.js_on_change("value", CustomJS(code="""
    console.log('range_slider: value=' + this.value, this.toString())
"""))
#Adding callback code
callback = CustomJS(args=dict(source=source, roll_mean_sld=sld),
                   code="""
     console.log('sld: value=' + this.value, this.toString())               
    const data = source.data;
    roll_mean_num = roll_mean_sld.value;
    
    x = data['x']
    y = data['y']
    sum=0
    for (let i = 0; i < x.length; i++) {
        y[i] = df['linear_interpolated'].rolling(roll_mean_num).mean() 
    }
      source.change.emit();
""")
source = ColumnDataSource(data=dict(x=x, y=y)) 
p.line('x', 'y', source=source, line_width=2,legend_label="rolling mean", color='green')

# Arrange plots and slider in layouts
layout = row(p, column(sld))
show(layout)